In [17]:
import numpy as np
import pandas as pd

df=pd.read_csv("/kaggle/input/resume-dataset/Resume/Resume.csv")
df.drop(["Resume_html", "ID"], axis=1, inplace=True)

df.head()





,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR


In [18]:
df.shape

(2484, 2)

In [19]:
df['Category'].value_counts()

Category
INFORMATION-TECHNOLOGY    120
BUSINESS-DEVELOPMENT      120
ADVOCATE                  118
CHEF                      118
ENGINEERING               118
ACCOUNTANT                118
FINANCE                   118
FITNESS                   117
AVIATION                  117
SALES                     116
BANKING                   115
HEALTHCARE                115
CONSULTANT                115
CONSTRUCTION              112
PUBLIC-RELATIONS          111
HR                        110
DESIGNER                  107
ARTS                      103
TEACHER                   102
APPAREL                    97
DIGITAL-MEDIA              96
AGRICULTURE                63
AUTOMOBILE                 36
BPO                        22
Name: count, dtype: int64

In [20]:
df['Resume_str'][0]

"         HR ADMINISTRATOR/MARKETING ASSOCIATE\n\nHR ADMINISTRATOR       Summary     Dedicated Customer Service Manager with 15+ years of experience in Hospitality and Customer Service Management.   Respected builder and leader of customer-focused teams; strives to instill a shared, enthusiastic commitment to customer service.         Highlights         Focused on customer satisfaction  Team management  Marketing savvy  Conflict resolution techniques     Training and development  Skilled multi-tasker  Client relations specialist           Accomplishments      Missouri DOT Supervisor Training Certification  Certified by IHG in Customer Loyalty and Marketing by Segment   Hilton Worldwide General Manager Training Certification  Accomplished Trainer for cross server hospitality systems such as    Hilton OnQ  ,   Micros    Opera PMS   , Fidelio    OPERA    Reservation System (ORS) ,   Holidex    Completed courses and seminars in customer service, sales strategies, inventory control, loss pr

In [21]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def text_preprocessing(data):
    stop_words = set(stopwords.words('english'))
    
    # Lowercase all text
    data['Resume_str'] = data['Resume_str'].apply(lambda x: x.lower())
    
    # Remove special characters and digits
    data['Resume_str'] = data['Resume_str'].apply(lambda x: re.sub('[^a-z\s]', ' ', x))
    
    # Remove URLs
    data['Resume_str'] = data['Resume_str'].apply(lambda x: re.sub(r'http\S+|www\S+', ' ', x))
    
    # Remove emails
    data['Resume_str'] = data['Resume_str'].apply(lambda x: re.sub(r'\S+@\S+', ' ', x))
    
    # Replace multiple spaces with a single space
    data['Resume_str'] = data['Resume_str'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())
    
    # Remove stopwords
    data['Resume_str'] = data['Resume_str'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    
    # Remove very short words (length 1)
    data['Resume_str'] = data['Resume_str'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))
    
    # Optional: keep slashes replaced (from your original)
    data['Resume_str'] = data['Resume_str'].apply(lambda x: x.replace("/", " "))
    
    # Add start and end tokens (optional — only if you plan seq2seq later)
    # For classification, you can comment this out

    
    return data

# Apply the function
data = text_preprocessing(df)
data.head()


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Resume_str,Category
0,hr administrator marketing associate hr admini...,HR
1,hr specialist us hr operations summary versati...,HR
2,hr director summary years experience recruitin...,HR
3,hr specialist summary dedicated driven dynamic...,HR
4,hr manager skill highlights hr skills hr depar...,HR


In [22]:
data['Resume_str'][0]

'hr administrator marketing associate hr administrator summary dedicated customer service manager years experience hospitality customer service management respected builder leader customer focused teams strives instill shared enthusiastic commitment customer service highlights focused customer satisfaction team management marketing savvy conflict resolution techniques training development skilled multi tasker client relations specialist accomplishments missouri dot supervisor training certification certified ihg customer loyalty marketing segment hilton worldwide general manager training certification accomplished trainer cross server hospitality systems hilton onq micros opera pms fidelio opera reservation system ors holidex completed courses seminars customer service sales strategies inventory control loss prevention safety time management leadership performance assessment experience hr administrator marketing associate hr administrator dec current company name city state helps devel

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(df['Category'])
df['Category'] = le.transform(df['Category'])


**from sklearn.preprocessing import LabelEncoder**

This imports the LabelEncoder class from scikit-learn.

LabelEncoder converts string (categorical) labels into integer IDs (0,1,2,...).

**le = LabelEncoder()**

This creates an instance of the encoder and stores it in the variable le.

At this point the encoder has no knowledge of your categories — it’s just an object ready to learn the mapping.

**le.fit(df['Category'])**

.fit(...) learns the mapping from label → integer using the data you pass.

df['Category'] should be a 1-D array-like (a pandas Series is fine).

Internally LabelEncoder finds the unique values in df['Category'], sorts them lexicographically, and stores them in le.classes_.

Example: if unique values are ['IT','Education','Healthcare'], after sorting it might be ['Education','Healthcare','IT'], and le.classes_ will be that array.

Important: fit only learns the mapping — it does not change df yet.

**df['Category'] = le.transform(df['Category'])**

.transform(...) replaces each label with the integer ID learned by .fit(...). It returns a NumPy array of integers.

Assigning that back to df['Category'] overwrites the original text labels with integer codes.

Example result: 'Education' → 0, 'Healthcare' → 1, 'IT' → 2 (depending on how they sorted).

If the Series contains a value that was not seen in fit, transform will raise a ValueError. (So make sure you fit on training data that includes all expected labels, or handle unknowns.)****

1️⃣** What TF-IDF Vectorizer does**
🔸 Concept

TF-IDF = Term Frequency – Inverse Document Frequency

It converts text into numerical vectors based on:

how often a word appears in a document (TF),

and how rare it is across all documents (IDF).

🔸 Example
Resume	Text	TF-IDF Vector (simplified)
1	"Python ML Developer"	[0.8, 0.4, 0.1]
2	"Web Developer"	[0.0, 0.7, 0.9]

Each number represents how important a word is in that document.

🔸 Output type:

A 2D sparse matrix (each row = document, each column = word weight)

Perfect for classical ML models like:

Logistic Regression

SVM

Naive Bayes

RandomForest

🧩 2️⃣** What Tokenizer does**
🔸 Concept

A Tokenizer (like Keras or Hugging Face) turns text into sequence(s) of integers —
each integer corresponds to a word/subword/token.

Example:

"I love machine learning" → [1, 4, 7, 8]


Then you can pad them to the same length:

[[1, 4, 7, 8],
 [5, 6, 0, 0]]

🔸 Output type:

A 2D NumPy array of integers

Ideal for deep learning models:

LSTM / GRU / RNN

CNN for text

Transformers / BERT etc.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=3000, ngram_range=(1,2), min_df=2)

tfidf.fit(df['Resume_str'])
requiredText = tfidf.transform(df['Resume_str'])


Example
💡 Step 1: Example Input

Imagine your dataset looks like this:

Resume (Text)
"Python developer skilled in ML and data analysis"
"Java backend developer experienced with Spring"
"Frontend developer with React and CSS skills"
🧾 Conceptually:
| Word ↓ / Resume → | Resume 1 | Resume 2 | Resume 3 |
| ----------------- | -------- | -------- | -------- |
| analysis          | 0.4      | 0.0      | 0.0      |
| backend           | 0.0      | 0.5      | 0.0      |
| css               | 0.0      | 0.0      | 0.5      |
| data              | 0.4      | 0.0      | 0.0      |
| developer         | 0.3      | 0.3      | 0.3      |
| experienced       | 0.0      | 0.6      | 0.0      |
| frontend          | 0.0      | 0.0      | 0.6      |
| java              | 0.0      | 0.4      | 0.0      |
| ml                | 0.4      | 0.0      | 0.0      |
| python            | 0.5      | 0.0      | 0.0      |
| react             | 0.0      | 0.0      | 0.4      |
| skilled           | 0.3      | 0.0      | 0.0      |
| spring            | 0.0      | 0.3      | 0.0      |


In [25]:
df

,Resume_str,Category
0,hr administrator marketing associate hr admini...,19
1,hr specialist us hr operations summary versati...,19
2,hr director summary years experience recruitin...,19
3,hr specialist summary dedicated driven dynamic...,19
4,hr manager skill highlights hr skills hr depar...,19
...,...,...
2479,rank sgt non commissioned officer charge briga...,6
2480,government relations communications organizati...,6
2481,geek squad agent professional profile support ...,6
2482,program director office manager summary highly...,6


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    requiredText,                  # Features
    df['Category'],                # Labels
    test_size=0.2,                 # 20% test data
    random_state=42,               # for reproducibility
    stratify=df['Category']        # keeps class distribution same
)


In [27]:
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)


Training data shape: (1987, 3000)
Testing data shape: (497, 3000)


In [28]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# knn = KNeighborsClassifier(
#     n_neighbors=18,   # number of neighbors (you can tune this)
#     metric='cosine'  # cosine distance works well for text data
# )
# knn.fit(X_train, y_train)
# y_pred = knn.predict(X_test,metric)
# print("✅ Accuracy:", accuracy_score(y_test, y_pred))
# print("\n🔍 Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))
# print("\n📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

🧩 What is “One-vs-Rest” (OvR or One-vs-All)?

👉 One-vs-Rest (OvR) is a strategy for handling multi-class classification problems
when your base algorithm can only handle binary classification (yes/no).

It means:

For each class, the model learns to separate that class (One) from all the other classes (Rest).

🎯 Example

Let’s say your resume dataset has 3 categories:

Resume Category	Meaning
0	Data Science
1	IT
2	Management

You want a classifier to predict which category a resume belongs to.

But KNN (and many other classifiers like Logistic Regression, SVM, etc.) are originally binary classifiers —
they only answer questions like:

“Is this a Data Science resume (Yes/No)?”

So how do we handle 3 classes?
That’s where One-vs-Rest comes in.

🧠 How One-vs-Rest Works

It trains one classifier per class.

So with 3 categories, you get 3 separate binary classifiers:

Classifier	What it learns
Model 1	“Is it Data Science (1) or not (0)?”
Model 2	“Is it IT (1) or not (0)?”
Model 3	“Is it Management (1) or not (0)?”

Then, when you give a new resume:

Each of the 3 classifiers outputs a probability score (confidence).

Example:

Data Science → 0.7
IT           → 0.2
Management   → 0.1


The highest score wins → final prediction = "Data Science".

⚙️ Visualization

Imagine three KNN classifiers running side by side:

        Resume
           |
  -----------------------------
  |           |              |
Model 1     Model 2       Model 3
(DS vs all) (IT vs all)  (Mgmt vs all)
  |           |              |
Score=0.7   Score=0.2     Score=0.1
           ↓
   Final Prediction → "Data Science"

💡 Why It’s Used

✅ Makes binary classifiers work for multi-class problems.
✅ Works well when classes are imbalanced.
✅ Easy to interpret and debug.
✅ Implemented automatically by many scikit-learn estimators.

🧩 Example Code — One-vs-Rest in Action
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

clf = OneVsRestClassifier(KNeighborsClassifier(n_neighbors=5, metric='cosine'))
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

What happens internally:

For every class in y_train, it trains a separate KNN model.

Each KNN learns “is it this class or not?”

Then it combines all results to pick the best match.

🧠 Analogy (to make it stick!)

Think of One-vs-Rest like an election:

Each class has its own candidate (e.g., “Data Science”, “IT”, “Management”).

Each classifier “votes” for how confident it is that the input belongs to its class.

The class with the highest confidence wins the election 🏆.

🧾 Quick Comparison
Strategy	Description	Example Use
One-vs-Rest (OvR)	1 model per class (class vs rest)	Logistic Regression, KNN
One-vs-One (OvO)	1 model for every pair of classes	SVM often uses this
Softmax (All-in-One)	Single model handles all classes together	Neural networks

In [29]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# clf = OneVsRestClassifier(knn)
# clf.fit(X_train, y_train)
# ypred = clf.predict(X_test)
# print("✅ Accuracy:", accuracy_score(y_test, y_pred))
# print("\n🔍 Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))
# print("\n📊 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


| Feature                   | KNN               | SVM (LinearSVC)            |
| ------------------------- | ----------------- | -------------------------- |
| Works on TF-IDF (sparse)? | ❌ Poor            | ✅ Excellent                |
| Speed                     | 🐢 Very slow      | ⚡ Very fast                |
| Generalization            | ❌ Overfits easily | ✅ Learns proper boundaries |
| Large dataset handling    | ❌ Struggles       | ✅ Scales well              |
| Distance computation      | ✅ Required        | ❌ Not needed               |


🧠 What is SVM (Support Vector Machine)?

SVM is a supervised machine learning algorithm used for classification and regression, but it shines in text classification because of how it handles high-dimensional TF-IDF data.

In [30]:
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (1987, 3000)
Test shape: (497, 3000)


In [31]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize SVM (multi-class One-vs-Rest)
model = OneVsRestClassifier(LinearSVC())

# Train
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n🔍 Classification Report:\n", classification_report(y_test, y_pred))


✅ Accuracy: 0.7364185110663984

🔍 Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.88      0.78        24
           1       0.64      0.75      0.69        24
           2       0.73      0.62      0.67        13
           3       0.54      0.37      0.44        19
           4       0.60      0.43      0.50        21
           5       0.75      0.43      0.55         7
           6       0.90      0.79      0.84        24
           7       0.78      0.78      0.78        23
           8       0.00      0.00      0.00         4
           9       0.68      0.88      0.76        24
          10       0.86      0.75      0.80        24
          11       0.83      0.86      0.84        22
          12       0.67      0.52      0.59        23
          13       0.91      0.95      0.93        21
          14       0.62      0.53      0.57        19
          15       0.76      0.79      0.78        24
          16       0.90